# Tuning Neural Networks with Normalization

## Load the data
First, let's run the below cell to import all the neccessary libraries and classes we will need.

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

We shall be working with the housing prices data. Data preprocessing already completed so we will focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken:

 - The data was split into the training, validate, and test sets
 - All the missing values in numeric columns were replaced by the median of those columns
 - All the missing values in catetgorical columns were replaced with the word 'missing'
 - All the categorical columns were one-hot encoded
 
Run the below cells to import the train, validate, and test sets:

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('X_train_numeric.csv')
X_test_numeric = pd.read_csv('X_test_numeric.csv')
X_val_numeric = pd.read_csv('X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('X_train_cat.csv')
X_test_cat = pd.read_csv('X_test_cat.csv')
X_val_cat = pd.read_csv('X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')
y_val = pd.read_csv('y_val.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

In [4]:
# Preview training data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build as Baseline Model
Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model.
In the cell below:
 - Add an input layer with `n_features` units
 - Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `relu` activation function)
 - Add an output layer with 1 unit and `linear` activation
 - Compile and fit the model

Here, we're calling `.shape` on our training data so that we can use the result as `n_features`, so we know how big to make our input layer.

In [5]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


In [6]:
#Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> **Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**

## Normalize the Input Data
 - Assign the column names of all numeric columns to `numeric_columns`
 - Instantiate a `StandardScaler`
 - Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names)
 - Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well
 - Use the provided to combine the scaled numerical and categorical features

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before):

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

 - Train the `normalized_input_model` on normalized input `(X_train)` and output `(y_train)`
 - Set a batch size of 32 and train for 150 epochs
 - Specify the `validation_data` argument as `(X_val, y_val)` Again, you may get some strange behavior.

In [9]:
# Train the model
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> **Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**

## Normalizing the output

Again, use `StandardScaler()` to:
 - Fit and transform `y_train`
 - Transform `y_val` and `y_test`

In [10]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

 - Train the normalized_model on normalized input `(X_train)` and output `(y_train_scaled)`
 - Set a batch size of 32 and train for 150 epochs
 - Specify the validation_data as `(X_val, y_val_scaled)`

In [11]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4899 - mse: 0.4899 - val_loss: 0.2043 - val_mse: 0.2043
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2203 - mse: 0.2203 - val_loss: 0.1570 - val_mse: 0.1570
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1762 - mse: 0.1762 - val_loss: 0.1484 - val_mse: 0.1484
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1578 - mse: 0.1578 - val_loss: 0.1842 - val_mse: 0.1842
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1416 - mse: 0.1416 - val_loss: 0.1387 - val_mse: 0.1387
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1266 - mse: 0.1266 - val_loss: 0.1414 - val_mse: 0.1414
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1203 - mse: 0.1203 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1082 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1455 - val_mse: 0.1455
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1446 - val_mse: 0.1446
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1453 - val_mse: 0.1453
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1475 - val_mse: 0.1475
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0205 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1571 - val_mse: 0.1571
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1576 - val_mse: 0.1576
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1570 - val_mse: 0.1570
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1572 - val_mse: 0.1572
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1581 - val_mse: 0.1581
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1591 - val_mse: 0.1591
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1584 - val_mse: 0.1584
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged.

Evaluate the model `(normalized_model)` on training data `(X_train and y_train_scaled)`

In [12]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 927us/step - loss: 0.0076 - mse: 0.0076


[0.0075514717027544975, 0.0075514717027544975]

Evaluate the model `(normalized_model)` on validate data (`X_val` and `y_val_scaled`)

In [13]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1610 - mse: 0.1610


[0.1610226184129715, 0.1610226184129715]

Since the output is normalized, the metric above is not interpretable. To remedy this:
 - Generate predictions on validate data (`X_val`)
 - Transform these predictions back to original scale using `ss_y`
 - Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred`

In [14]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

9/9 [==============================] - 0s 2ms/step


31533.7014722393

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.)

## Using Weight Initializers
We will to use alternative initialization and optimization strategies. At the end, we'll then select the model which we believe performs the best.

### He Initialization
In the cell below, sepcify the following in the first hidden layer:

 - 100 units
 - `relu` activation
 - input_shape
 - kernel_initializer=`he_normal`

In [15]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.4965 - mse: 0.4965 - val_loss: 0.2000 - val_mse: 0.2000
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2614 - mse: 0.2614 - val_loss: 0.1843 - val_mse: 0.1843
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1854 - mse: 0.1854 - val_loss: 0.1373 - val_mse: 0.1373
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1682 - mse: 0.1682 - val_loss: 0.1370 - val_mse: 0.1370
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1451 - mse: 0.1451 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1297 - mse: 0.1297 - val_loss: 0.1430 - val_mse: 0.1430
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1189 - mse: 0.1189 - val_loss: 0.1430 - val_mse: 0.1430
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1074 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1431 - val_mse: 0.1431
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1415 - val_mse: 0.1415
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1406 - val_mse: 0.1406
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1399 - val_mse: 0.1399
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0210 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1444 - val_mse: 0.1444
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1451 - val_mse: 0.1451
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1463 - val_mse: 0.1463
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1445 - val_mse: 0.1445
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1485 - val_mse: 0.1485
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1451 - val_mse: 0.1451
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`)

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 882us/step - loss: 0.0083 - mse: 0.0083


[0.008272567763924599, 0.008272567763924599]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1466 - mse: 0.1466


[0.1465965211391449, 0.1465965211391449]

### Lecun Initialization
In the cell below, sepcify the following in the first hidden layer:

 - 100 units
 - `relu` activation
 - input_shape
 - kernel_initializer=`lecun_normal`

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5029 - mse: 0.5029 - val_loss: 0.2159 - val_mse: 0.2159
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2367 - mse: 0.2367 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1815 - mse: 0.1815 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1565 - mse: 0.1565 - val_loss: 0.1207 - val_mse: 0.1207
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1370 - mse: 0.1370 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1266 - mse: 0.1266 - val_loss: 0.1117 - val_mse: 0.1117
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1121 - mse: 0.1121 - val_loss: 0.1089 - val_mse: 0.1089
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1091 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0204 - mse: 0.0204 - val_loss: 0.0930 - val_mse: 0.0930
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0205 - mse: 0.0205 - val_loss: 0.0937 - val_mse: 0.0937
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.0930 - val_mse: 0.0930
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0198 - mse: 0.0198 - val_loss: 0.0923 - val_mse: 0.0923
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0196 - mse: 0.0196 - val_loss: 0.0924 - val_mse: 0.0924
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0189 - mse: 0.0189 - val_loss: 0.0925 - val_mse: 0.0925
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0191 - mse: 0.0191 - val_loss: 0.0923 - val_mse: 0.0923
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0186 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.0921 - val_mse: 0.0921
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.0919 - val_mse: 0.0919
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.0922 - val_mse: 0.0922
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.0935 - val_mse: 0.0935
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.0920 - val_mse: 0.0920
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`)

In [19]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0078 - mse: 0.0078


[0.007761810906231403, 0.007761810906231403]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`)

In [20]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0924 - mse: 0.0924


[0.09242885559797287, 0.09242885559797287]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

### RMSprop
Compile the `rmsprop_model` with:
 - `rmsprop` as the optimizer
 - track `mse` as the `loss` and `metric`

In [21]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3260 - mse: 0.3260 - val_loss: 0.1366 - val_mse: 0.1366
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1652 - mse: 0.1652 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1237 - mse: 0.1237 - val_loss: 0.1270 - val_mse: 0.1270
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1010 - mse: 0.1010 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0752 - mse: 0.0752 - val_loss: 0.1066 - val_mse: 0.1066
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0727 - mse: 0.0727 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0572 - mse: 0.0572 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0420 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.0997 - val_mse: 0.0997
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0991 - val_mse: 0.0991
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1066 - val_mse: 0.1066
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0996 - val_mse: 0.0996
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.0865 - val_mse: 0.0865
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.0943 - val_mse: 0.0943
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0060 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0876 - val_mse: 0.0876
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0874 - val_mse: 0.0874
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0896 - val_mse: 0.0896
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0970 - val_mse: 0.0970
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.0943 - val_mse: 0.0943
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0913 - val_mse: 0.0913
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.0913 - val_mse: 0.0913
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`)

In [22]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0038 - mse: 0.0038


[0.0037923636846244335, 0.0037923636846244335]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`)

In [23]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0813 - mse: 0.0813


[0.08131416887044907, 0.08131416887044907]

### Adam
Compile the `adam_model` with:
 - `Adam` as the optimizer
 - track `mse` as the `loss` and `metric`

In [24]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3911 - mse: 0.3911 - val_loss: 0.1581 - val_mse: 0.1581
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1512 - mse: 0.1512 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1074 - mse: 0.1074 - val_loss: 0.1127 - val_mse: 0.1127
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0842 - mse: 0.0842 - val_loss: 0.1450 - val_mse: 0.1450
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0630 - mse: 0.0630 - val_loss: 0.0976 - val_mse: 0.0976
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0492 - mse: 0.0492 - val_loss: 0.1359 - val_mse: 0.1359
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0416 - mse: 0.0416 - val_loss: 0.1312 - val_mse: 0.1312
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0334 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1244 - val_mse: 0.1244
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1260 - val_mse: 0.1260
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.1245 - val_mse: 0.1245
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0015 - mse: 0.0015 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1270 - val_mse: 0.1270
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0022 - mse: 0

Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0034 - mse: 0.0034 - val_loss: 0.1127 - val_mse: 0.1127
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0033 - mse: 0.0033 - val_loss: 0.1098 - val_mse: 0.1098
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1090 - val_mse: 0.1090
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1228 - val_mse: 0.1228
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`)

In [25]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 923us/step - loss: 3.3118e-04 - mse: 3.3118e-04


[0.00033117944258265197, 0.00033117944258265197]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`)

In [26]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1093 - mse: 0.1093


[0.10934751480817795, 0.10934751480817795]

### Select a Final Model
Now, we select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [27]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 2ms/step - loss: 0.1775 - mse: 0.1775


[0.1775279939174652, 0.1775279939174652]

 - Generate predictions on test data (`X_test`)
 - Transform these predictions back to original scale using `ss_y`
 - Now you can calculate the `RMSE` in the original units with `y_test` and `y_test_pred`

In [28]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

5/5 [==============================] - 0s 1ms/step


33110.44228080478

## Summary

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.